In [1]:
from grandma_stock_valuation import FileLogger, YahooDataLoader, batchValuation, addCashPortfolio, allocatePortfolio
import time

logger = FileLogger()
logPrint = logger.log_pandas


### Refresh data of each instrument

Query data of the selected instruments from Yahoo Finance.

With `save=True`, a `__data__` folder will be created to store the queried data.

In [2]:
d_instrument = {
    'IVV':'SP500',
    '3073.HK':'Greater China',
    'VPL':'Developed Asia-Pacific',
    'IEV':'Europe',
    'ASEA':'SE Asia',

    'AAXJ':'Asia ex Japan',
    'EZU':'Eurozone'
}

d_instrument_data = {}
for ticker, name in d_instrument.items():
    logPrint(f"{ticker}: {name}")

    # Refresh data
    yahoo = YahooDataLoader(ticker, date_start='2012-01-01', verbose=0, printfunc=logPrint) # data_start can be removed, if this instrument already has historical data under __data__ folder.
    df = yahoo.queryEOD(save=True)

    d_instrument_data[ticker] = df
    time.sleep(2)



2022-02-22 08:08:16,381 INFO IVV: SP500
2022-02-22 08:08:18,695 INFO 3073.HK: Greater China
2022-02-22 08:08:20,890 INFO VPL: Developed Asia-Pacific
2022-02-22 08:08:23,131 INFO IEV: Europe
2022-02-22 08:08:25,414 INFO ASEA: SE Asia
2022-02-22 08:08:27,603 INFO AAXJ: Asia ex Japan
2022-02-22 08:08:29,819 INFO EZU: Eurozone


### Valuate the instruments

With `save_result=True`, a `__output__` folder will be created to store valuation metrics in a .csv file.

A figure of each instrument, which is the price chart with fitted trend line, will be stored under `__output__/images`.

In [3]:
df_valuation_metrics, d_fig = batchValuation(d_instrument_data, verbose=0, save_result=True, printfunc=logPrint)

df_valuation_metrics

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years
0,IVV,0.970999,10.0,0.134356,436.070007,410.53761,0.062193,0.462896
1,3073.HK,0.916884,10.00274,0.082724,51.220001,52.423557,-0.022958,-0.18992
2,VPL,0.890518,10.0,0.067472,74.959999,77.122058,-0.028034,-0.189153
3,IEV,0.781296,10.0,0.052801,51.939999,49.197782,0.055739,1.055628
4,ASEA,0.346128,10.0,0.016464,15.95,14.34943,0.111542,6.774757
5,AAXJ,0.851064,10.0,0.061744,81.330002,81.351094,-0.000259,-0.001601
6,EZU,0.796705,10.0,0.061367,46.470001,45.659508,0.017751,0.289258


### Construct a portfolio

Based on the valuation, the suggested allocation of the insturments in your portfolio.

In [4]:
instruments_select = ['IVV', '3073.HK']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)].copy()

df_portfolio['portfolio_allocation'] = allocatePortfolio(df_portfolio['over_value_years'], with_cash=False)

df_portfolio

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,portfolio_allocation
0,IVV,0.970999,10.0,0.134356,436.070007,410.53761,0.062193,0.462896,0.342355
1,3073.HK,0.916884,10.00274,0.082724,51.220001,52.423557,-0.022958,-0.18992,0.657645


Optionally, you can add cash as part of your portfolio. This allows the model to realize profit when most instruments are over-valued.

In [6]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'IEV', 'ASEA']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)].copy()

df_portfolio_with_cash = addCashPortfolio(df_portfolio)

df_portfolio_with_cash['portfolio_allocation'] = allocatePortfolio(df_portfolio_with_cash['over_value_years'], with_cash=True)

df_portfolio_with_cash

,ticker,r2_train,train_years,annualized_return,currenct_price,fair_price,over_value_range,over_value_years,portfolio_allocation
0,IVV,0.970999,10.0,0.134356,436.070007,410.53761,0.062193,0.462896,0.109109
1,3073.HK,0.916884,10.00274,0.082724,51.220001,52.423557,-0.022958,-0.189920,0.310086
2,VPL,0.890518,10.0,0.067472,74.959999,77.122057,-0.028034,-0.189153,0.309705
3,IEV,0.781296,10.0,0.052801,51.939999,49.197782,0.055739,1.055628,0.042266
4,ASEA,0.346128,10.0,0.016464,15.95,14.34943,0.111542,6.774757,0.000004
5,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.228830
